#### Tokenization with Stanford Tokenizer.
Please check, that Stanford Server is running:    
[Start server and/or check that it is running](http://localhost:8888/notebooks/MLClassification/tokenizer/arabic/stanford/Start%20Stanford%20Server%20from%20Python.ipynb#2)    

It is assumed here, that all source text files will be placed in the folder ~/MLClassificationData.    
Original and resulting (tokenized) files will have the same names and will be placed in the same file's tree, under the same root folder:
- original files - under subfolder _source_, 
- tokenized files - under subfolder _target_.

_**Note:**_ you should change value of variable _root_ to set actual name of the root folder.

In [ ]:
import numpy
import random
import glob
import os
import shutil
import datetime
from collections import OrderedDict
from collections import namedtuple
from random import shuffle
from pathlib import Path
from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser(url='http://localhost:9005', tagtype='pos')

# Change value of this variable 
root = "test"
homePath = str(Path.home()) + "/MLClassificationData"
inPath = homePath + "/" + root + "/source"

def tokenizeData(path):
    curDir = os.getcwd()
    os.chdir(path);
    for ff in glob.glob("*"):
        if os.path.isdir(ff):
            dPath = path + "/" + ff
            tPath = dPath.replace("source","target")
            if os.path.exists(tPath):
                shutil.rmtree(tPath)
            os.mkdir(tPath)
            tokenizeData(dPath)
            continue
        fPath = path + "/" + ff
        result = ''
        q = 0;        
        qt = 0;
        ds = datetime.datetime.now()
        with open(fPath, 'r', encoding='UTF-8') as f:            
            for line in f:
                q += 1
                line = line.replace('\r', '').replace('\n', '')
                if len(line) == 0:
                    continue
                toks = line.split()
                print ("                                                                 ", end='\r')
                print ("Load %d line, len: %d, toks: %d"%(q, len(line), len(toks)), end='\r')               
                if len(toks) == 0:
                    continue
                qt += len(toks)
                tArr = parser.tag(line.split())
                result += joinTokens(tArr).strip() + "\n"
        de = datetime.datetime.now()
        print ("File %s (%d lines, %d tokens) is tokenized in %d sec."%(fPath, q, qt, (de-ds).total_seconds()))
        f.close()
        outPath = fPath.replace("source", "target")
        outFile = open(outPath, 'w', encoding='UTF-8')
        outFile.write(result.strip())
        outFile.close()

def joinTokens(tArr):
    toks = [x[0] for x in tArr]
    tags = [x[1] for x in tArr]
    result = ''
    for i in range(len(tArr)):        
        ftok = ''
        if tags[i] == "DT" or tags[i] == "IN" or tags[i] == "PUNC" or tags[i] == "CD" or tags[i] == 'PRP$':
            continue
        else:
            if tags[i].startswith("DT"):
                ftok = toks[i][2:]
            else:
                ftok = toks[i]
            result += ftok
    return result

fds = datetime.datetime.now()
tokenizeData(inPath)
fde = datetime.datetime.now()
print ("Tokenization complited in %d sec"%((fde-fds).total_seconds()))